In [1]:
import cv2
import numpy as np # We'll be storing our data as numpy arrays
import os # For handling directories
from PIL import Image # For handling the images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg # Plotting

In [2]:
%matplotlib inline

In [3]:
pwd

'D:\\gesture-recognition'

In [4]:
lookup = dict()
reverselookup = dict()
count = 0
for j in os.listdir('data/'):
    if not j.startswith('.'): # If running this code locally, this is to 
                              # ensure you aren't reading in hidden folders
        lookup[j] = count
        reverselookup[count] = j
        count = count + 1


In [5]:
lookup

{'fist': 0, 'L': 1, 'none': 2, 'ok': 3, 'palm': 4, 'peace': 5, 'thumbsup': 6}

In [6]:
reverselookup

{0: 'fist', 1: 'L', 2: 'none', 3: 'ok', 4: 'palm', 5: 'peace', 6: 'thumbsup'}

In [7]:
x_data = []
y_data = []
datacount = 0 
 # Loop over the ten top-level folders
for j in os.listdir('data/'):
    if not j.startswith('.'): # Again avoid hidden folders
        count = 0 # To tally images of a given gesture
        for k in os.listdir('data/' + j + '/'):
                            # Loop over the images
            img = Image.open('data/' + j + '/' + k)
                            # Read in and convert to greyscale
            arr = np.array(img)
            x_data.append(arr) 
            count = count + 1
        y_values = np.full((count, 1), lookup[j]) 
        y_data.append(y_values)
        datacount = datacount + count

In [8]:
x_data = np.array(x_data, dtype = 'float32')
y_data = np.array(y_data)
y_data = y_data.reshape(datacount, 1)

In [9]:
x_data.shape

(2100, 300, 300)

In [10]:
y_data.shape

(2100, 1)

In [11]:
from keras.utils import to_categorical
y_data = to_categorical(y_data)

Using TensorFlow backend.
d:\mltensor\lib\site-packages\tensorflow\python\framework\dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\mltensor\lib\site-packages\tensorflow\python\framework\dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\mltensor\lib\site-packages\tensorflow\python\framework\dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\mltensor\lib\site-packages\tensorflow\python\framework\dtypes.py:496: FutureWarning: Passing (type, 1) or '1type' as

In [12]:
y_data.shape

(2100, 7)

In [13]:
x_data = x_data.reshape((datacount, 300, 300, 1))
x_data /= 255
x_data.shape

(2100, 300, 300, 1)

In [14]:
from sklearn.model_selection import train_test_split
x_train,x_further,y_train,y_further = train_test_split(x_data,y_data,test_size = 0.1)
x_validate,x_test,y_validate,y_test = train_test_split(x_further,y_further,test_size = 0.6)

In [15]:
from keras import layers
from keras import models

In [16]:
model=models.Sequential()
model.add(layers.Conv2D(32, (5, 5), strides=(2, 2), activation='relu', input_shape=(300, 300,1))) 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18496)             0         
__________

In [18]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=64, verbose=1, validation_data=(x_validate, y_validate))

Train on 1890 samples, validate on 84 samples
Epoch 1/10
1890/1890 [==============================] - 66s 35ms/step - loss: 0.8038 - acc: 0.7640 - val_loss: 0.1302 - val_acc: 0.9524
Epoch 2/10
1890/1890 [==============================] - 73s 39ms/step - loss: 0.0901 - acc: 0.9751 - val_loss: 0.0883 - val_acc: 0.9762
Epoch 3/10
1890/1890 [==============================] - 70s 37ms/step - loss: 0.0162 - acc: 0.9958 - val_loss: 0.2047 - val_acc: 0.9524
Epoch 4/10
1890/1890 [==============================] - 66s 35ms/step - loss: 0.0102 - acc: 0.9963 - val_loss: 0.0415 - val_acc: 0.9881
Epoch 5/10
1890/1890 [==============================] - 65s 35ms/step - loss: 6.0277e-05 - acc: 1.0000 - val_loss: 0.0363 - val_acc: 0.9881
Epoch 6/10
1890/1890 [==============================] - 65s 34ms/step - loss: 8.9510e-06 - acc: 1.0000 - val_loss: 0.0470 - val_acc: 0.9881
Epoch 7/10
1890/1890 [==============================] - 77s 41ms/step - loss: 1.5860e-06 - acc: 1.0000 - val_loss: 0.0425 - val_ac

In [19]:
[loss, acc] = model.evaluate(x_validate,y_validate,verbose=1)
print("Accuracy:" + str(acc))

84/84 [==============================] - 1s 11ms/step
Accuracy:0.9880952380952381


In [20]:
new = Image.open('test/4.png')
a = np.array(new)
a = a/255
a = a.reshape(1, 300, 300, 1)

In [21]:
result = model.predict(a)

In [22]:
result

array([[4.48361671e-17, 5.71457443e-21, 3.33096848e-21, 1.00000000e+00,
        9.00522538e-12, 4.96676593e-13, 1.35628256e-17]], dtype=float32)

In [23]:
lookup

{'fist': 0, 'L': 1, 'none': 2, 'ok': 3, 'palm': 4, 'peace': 5, 'thumbsup': 6}

In [24]:
label = {}

In [25]:
for y, x in lookup.items():
    label[x] = y

In [26]:
label

{0: 'fist', 1: 'L', 2: 'none', 3: 'ok', 4: 'palm', 5: 'peace', 6: 'thumbsup'}

In [27]:
l = np.where(result[0] == np.amax(result[0]))

In [28]:
print(label[l[0][0]])

ok


In [29]:
print(result[0][l][0])

1.0


In [30]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
